In [0]:

#En la carpeta "data_cp_flights" hay un archivo Parquet, comprimido con Snappy, en adelante, la tabla "flights". ¿Qué cantidad de columnas posee?

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("consulta-parquet").getOrCreate()
parquet_path = "dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/raw_flight_data_snappy-1.parquet"
df = spark.read.parquet(parquet_path)

In [0]:
#Quitar de la tabla "flights" los registros que tengan valores nulos o faltantes en los campos "ArrDelay" y "DepDelay". Luego contestar las siguientes preguntas:


from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crea una sesión de Spark
spark = SparkSession.builder.appName("consulta-parquet").getOrCreate()

# Ruta al archivo Parquet
parquet_path = "dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/raw_flight_data_snappy-1.parquet"

# Lee el archivo Parquet y crea un DataFrame
df = spark.read.parquet(parquet_path)

# Asigna un nombre temporal al DataFrame
df.createOrReplaceTempView("mi_df_temporal")

# Muestra el nombre del DataFrame
print("Nombre del DataFrame:", spark.sql("SELECT * FROM mi_df_temporal LIMIT 1"))

# Filtra los registros que no tienen valores nulos en las columnas especificadas
mi_tabla_sin_nulos = spark.sql("SELECT * FROM mi_df_temporal WHERE ArrDelay IS NOT NULL AND DepDelay IS NOT NULL")

# Muestra el DataFrame resultante
mi_tabla_sin_nulos.show()
#-----------------------------------------------------------------------------------
#¿Cuál es la tupla de aeropuertos, con mayor cantidad de vuelos entre sí? Nota: Es posible tomar el nombre del aeropuerto desde el archivo "airports.json" ubicado en la carpeta "data_cp_airports", donde "airport_id" se puede #relacionar con "OriginAirportID" y "DestAirportID" de la tabla "flights" 


# Agrupa por pares de aeropuertos y cuenta la cantidad de vuelos
pairs_count = mi_tabla_sin_nulos.groupBy("OriginAirportID", "DestAirportID").count()

# Encuentra el par con la mayor cantidad de vuelos
max_flight_pair = pairs_count.orderBy(col("count").desc()).first()

# Imprime la información del par con la mayor cantidad de vuelos
print("Aeropuertos con mayor cantidad de vuelos:")
print("OriginAirportID:", max_flight_pair["OriginAirportID"])
print("DestAirportID:", max_flight_pair["DestAirportID"])
print("Cantidad de vuelos:", max_flight_pair["count"])


Nombre del DataFrame: DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int]
+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        30|        4|     UA|          13930|        10721|      -3|      -7|
|        30|        4|     UA|          11618|        12892|      -1|     -28|
|        30|        4|     UA|          12892|        14771|      89|      84|
|        30|        4|     UA|          13930|        15304|      -2|      -2|
|        30|        4|     UA|          11618|        12266|       7|      23|
|        30|        4|     UA|          12266|        13851|      -2|      -7|
|        30|        4|     UA|          13930|        12953|      72|      65|
|        30|        4|     UA|          14771|        11

In [0]:
#consultar nombres ( Es posible tomar el nombre del aeropuerto desde el archivo "airports.csv", donde "airport_id" se puede relacionar con "OriginAirportID" y "DestAirportID" de la tabla "flights")

from pyspark.sql import SparkSession

# Crea una sesión de Spark
spark = SparkSession.builder.appName("consulta-sql").getOrCreate()

# Lee el archivo CSV y crea un DataFrame llamado df1
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/airports-1.csv")

# Crea una vista temporal con el nombre "mi_df_temporal2" para el DataFrame df1
df1.createOrReplaceTempView("mi_df_temporal2")

# Número que quieres buscar en la columna "airport_id"
numero_ingresado = 12173

# Ejecuta la consulta SQL para seleccionar todas las columnas donde "airport_id" es igual al número ingresado
resultado = spark.sql(f"SELECT * FROM mi_df_temporal2 WHERE airport_id = {numero_ingresado}")

# Muestra el resultado
resultado.show()



+----------+--------+-----+--------------------+
|airport_id|    city|state|                name|
+----------+--------+-----+--------------------+
|     12173|Honolulu|   HI|Honolulu Internat...|
+----------+--------+-----+--------------------+



In [0]:
#Si consideramos que cuando el campo ArrDelay es mayor a 25, el vuelo está demorado, contestar: ¿Cuál es la tupla de aeropuertos, con mayor cantidad de vuelos demorados entre sí?

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crea una sesión de Spark
spark = SparkSession.builder.appName("consulta-parquet").getOrCreate()

# Ruta al archivo Parquet
parquet_path = "dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/raw_flight_data_snappy-1.parquet"

# Lee el archivo Parquet y crea un DataFrame
df = spark.read.parquet(parquet_path)

# Asigna un nombre temporal al DataFrame
df.createOrReplaceTempView("mi_df_temporal")

# Filtra los vuelos demorados (ArrDelay > 25)
vuelos_demorados = spark.sql("SELECT * FROM mi_df_temporal WHERE ArrDelay > 25")

# Agrupa por pares de aeropuertos y cuenta la cantidad de vuelos demorados
pares_demorados_count = vuelos_demorados.groupBy("OriginAirportID", "DestAirportID").count()

# Encuentra el par con la mayor cantidad de vuelos demorados
max_demorados_pair = pares_demorados_count.orderBy(col("count").desc()).first()

# Imprime la información del par con la mayor cantidad de vuelos demorados
print("Aeropuertos con mayor cantidad de vuelos demorados:")
print("OriginAirportID:", max_demorados_pair["OriginAirportID"])
print("DestAirportID:", max_demorados_pair["DestAirportID"])
print("Cantidad de vuelos demorados:", max_demorados_pair["count"])


Aeropuertos con mayor cantidad de vuelos demorados:
OriginAirportID: 12892
DestAirportID: 14771
Cantidad de vuelos demorados: 2375


In [0]:
#Si consideramos que cuando el campo ArrDelay es mayor a 25, el vuelo está demorado, contestar: ¿Cuál es el aeropuerto con el mayor promedio de demora en arribos, teniendo en cuenta el de orígen?

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crea una sesión de Spark
spark = SparkSession.builder.appName("consulta-parquet").getOrCreate()

# Ruta al archivo Parquet
parquet_path = "dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/raw_flight_data_snappy-1.parquet"

# Lee el archivo Parquet y crea un DataFrame
df = spark.read.parquet(parquet_path)

# Filtra los vuelos demorados (ArrDelay > 25)
vuelos_demorados = df.filter(col("ArrDelay") > 25)

# Agrupa por aeropuerto de origen y calcula el promedio de demora en arribos
promedio_demora_por_aeropuerto = vuelos_demorados.groupBy("OriginAirportID").agg({"ArrDelay": "avg"})

# Encuentra el aeropuerto con el mayor promedio de demora en arribos
max_promedio_aeropuerto = promedio_demora_por_aeropuerto.orderBy(col("avg(ArrDelay)").desc()).first()

# Imprime la información del aeropuerto con el mayor promedio de demora en arribos
print("Aeropuerto con el mayor promedio de demora en arribos:")
print("OriginAirportID:", max_promedio_aeropuerto["OriginAirportID"])
print("Promedio de demora en arribos:", max_promedio_aeropuerto["avg(ArrDelay)"])


Aeropuerto con el mayor promedio de demora en arribos:
OriginAirportID: 12173
Promedio de demora en arribos: 94.41454272863568


In [0]:
#Si consideramos que cuando el campo ArrDelay es mayor a 25, el vuelo está demorado, contestar: ¿Cuál es la cantidad de vuelos demorados?

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crea una sesión de Spark
spark = SparkSession.builder.appName("consulta-parquet").getOrCreate()

# Ruta al archivo Parquet
parquet_path = "dbfs:/FileStore/shared_uploads/alejodiezgomez@gmail.com/raw_flight_data_snappy-1.parquet"

# Lee el archivo Parquet y crea un DataFrame
df = spark.read.parquet(parquet_path)

# Filtra los vuelos demorados (ArrDelay > 25)
vuelos_demorados = df.filter(col("ArrDelay") > 25)

# Cuenta la cantidad de vuelos demorados
cantidad_vuelos_demorados = vuelos_demorados.count()

# Imprime la cantidad de vuelos demorados
print("Cantidad de vuelos demorados:", cantidad_vuelos_demorados)


Cantidad de vuelos demorados: 380502
